In [14]:
"""
데이터 불러오기
"""

import pandas as pd
import numpy as np


data = pd.read_csv("sentiment_multiple_data.csv", encoding="cp949")

In [95]:
"""
데이터 라벨 정의 (정수형)
"""

data.loc[(data['상황'] == "fear"), '상황'] = 0  #공포 => 0
data.loc[(data['상황'] == "surprise"), '상황'] = 1  #놀람 => 1
data.loc[(data['상황'] == "angry"), '상황'] = 2  #분노 => 2
data.loc[(data['상황'] == "sadness"), '상황'] = 3  #슬픔 => 3
data.loc[(data['상황'] == "neutral"), '상황'] = 4  #중립 => 4
data.loc[(data['상황'] == "happiness"), '상황'] = 5  #행복 => 5
data.loc[(data['상황'] == "disgust"), '상황'] = 6  #혐오 => 6

In [96]:
"""
데이터 라벨 정의 (문자열)
"""
label_mapping = {
    0: "fear",
    1: "surprise",
    2: "angry",
    3: "sadness",
    4: "neutral",
    5: "happiness",
    6: "disgust"
}

In [76]:
"""
데이터 변환하기
"""
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    # data = []
    # data.append(ques)
    # data.append(str(label))
    data_dict = {"labels": np.int32(str(label)), "text": ques}

    data_list.append(data_dict)

In [77]:
"""
데이터 변환 확인
"""
print(data_list[0])

{'labels': np.int32(5), 'text': '헐! 나 이벤트에 당첨 됐어.'}


In [78]:
"""
토크나이저 불러오기
"""
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

In [79]:
"""
토큰화 함수 정의
"""
def tokenize_function(sample):
    return tokenizer(
        sample["text"],  # 텍스트 데이터
        # padding=True,  # 패딩 적용
        padding="max_length",  # 최대 길이로 패딩
        truncation=True,  # 최대 길이를 넘어가는 데이터는 자름
        return_tensors="pt"  # 파이토치 텐서로 반환
    )

In [83]:
"""
Dataset 으로 만들어 주기
"""
from datasets import Dataset

dataset = Dataset.from_list(data_list)


In [84]:
"""
토크나이저 사용하기
"""

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,  # 병렬 처리 활성화
    remove_columns=["text"])  # 원본 데이터에서 text 컬럼 제거 (메모리 최적화)

Map: 100%|██████████| 19374/19374 [00:04<00:00, 4753.27 examples/s]


In [88]:
"""
DatasetDict 로 만들어주기
훈련 데이터와 테스트 데이터로 나누기
"""

tokenized_sentiment_dataset = tokenized_dataset.train_test_split(
    test_size=0.2) # 테스트 데이터 비율

In [89]:
"""
DatasetDict 확인하기
"""
print(tokenized_sentiment_dataset)


DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 15499
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3875
    })
})


In [90]:
"""
데이터 셋 분리해서 준비하기
"""

# 학습용 데이터셋
train_dataset = tokenized_sentiment_dataset["train"]

# 테스트용 데이터셋
test_dataset = tokenized_sentiment_dataset["test"]

In [91]:
"""
분리된 데이터 셋 확인하기
"""

# 학습용 데이터셋 확인
print(train_dataset[0])

# 테스트용 데이터셋 확인
print(test_dataset[0])

{'labels': 2, 'input_ids': [2, 3093, 54, 3610, 517, 5938, 1282, 3097, 6844, 54, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [93]:
"""
예측값과 실제 레이블 간의 정확도 계산
"""

from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    # 튜플에서 예측값(predictions)과 실제값(labels) 추출
    predictions, labels = eval_pred

    # 예측값의 가장 높은 확률(로짓) 인덱스를 가져옴
    # 각 샘플에 대해 argmax를 사용하여 클래스 예측
    predictions = np.argmax(predictions, axis=1)

    # 정확도(accuracy) 계산
    return {
        "accuracy": float(
            accuracy_score(labels, predictions)
        )
    }

In [99]:
"""
BERT 분류기 사용하기
"""

from transformers import BertForSequenceClassification

clf_model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [100]:
"""
분류기 학습시키기
"""

from transformers import TrainingArguments

# 하이처 파라미터 지정
training_args = TrainingArguments(
    output_dir="../sentiment_classification/sentiment_multiple",  # 출력 디렉토리
    num_train_epochs=3,  # 학습할 에포크 수
    eval_strategy="steps",  # 평가 전략 : 에포크마다 검증 데이터셋에 대한 평가 지표 출력
    eval_steps=500,  # 500 스텝마다 평가
    per_device_train_batch_size=8,  # 학습에 사용할 배치
    # 사이즈
    per_device_eval_batch_size=16,  # 평가에 사용할 배치 사이즈
    warmup_steps=200,  # warmup 스텝 수
    weight_decay=0.01,  # 가중치 감쇠
    remove_unused_columns=False,  # 사용하지 않는 컬럼 제거
)

In [101]:
"""
Trainer 불러오기
"""
from transformers import Trainer

clf_trainer = Trainer(
    model=clf_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [103]:
"""
gpu 사용하기
"""

import torch

print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}")

# cuda 우선 사용
# mps 사용 가능하면 mps 사용
# 디바이스 우선순위 설정: CUDA → MPS → CPU
device_name = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_built() and torch.backends.mps.is_available()
    else "cpu"
)

device = torch.device(device_name)

clf_model.to(device)

MPS 장치를 지원하도록 build가 되었는가? True
MPS 장치가 사용 가능한가? True


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-

In [104]:
"""
clf Trainer 학습시키기
"""

clf_trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 